In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("cs109a_hw7.ipynb")

# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science

## Homework 7: Fairness and Bias


**Harvard University**<br/>
**Fall 2022**<br/>
**Instructors**: Pavlos Protopapas and Natesh Pillai

<hr style="height:2pt">

In [ ]:
# RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/"
    "themes/static/css/cs109.css"
).text
HTML(styles)

### Import Libraries

In [ ]:
import random
random.seed(112358)

import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import (
    accuracy_score, confusion_matrix, roc_curve, roc_auc_score
)                           
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

%matplotlib inline

<hr>

#### Instructions
- To submit your assignment follow the instructions given in Canvas.
- Plots should be legible and interpretable without having to refer to the code that generated them, including labels for the $x$- and $y$-axes as well as a descriptive title and/or legend when appropriate.
- When asked to interpret a visualization, do not simply describe it (e.g., "the curve has a steep slope up"), but instead explain what you think the plot *means*.
- The use of 'hard-coded' values to try and pass tests rather than solving problems programmatically will not receive credit.
- The use of *extremely* inefficient or error-prone code (e.g., copy-pasting nearly identical commands rather than looping) may result in only partial credit.
- We have tried to include all the libraries you may need to do the assignment in the imports cell provided below. Please get course staff approval before importing any additional 3rd party libraries.
- Enable scrolling output on cells with very long output.
- Feel free to add additional code or markdown cells as needed.
- Ensure your code runs top to bottom without error and passes all tests by restarting the kernel and running all cells. This is how the notebook will be evaluated (note that this can take a few minutes). 
- **You should do a "Restart Kernel and Run All Cells" before submitting to ensure (1) your notebook actually runs and (2) all output is visible**

<hr>

<a id="contents"></a>

## Notebook contents

- [**Required reading and data description**](#intro)


- [**Question 1: Data exploration [26 pts]**](#part1)


- [**Question 2: Baseline modeling [21 pts]**](#part2)


- [**Question 3: Predicting without using `race` [19 pts]**](#part3)


- [**Question 4: Classification thresholds and fairness [34 pts]**](#part4)

<a id="intro"></a>

## Required reading and data description

[Return to contents](#contents)

### Required reading

The required readings for this homework are two articles that report on the effectiveness of and issues surrounding the COMPAS recidivism algorithm used in U.S. courts. One article is published by ProPublica and the other by the MIT Technology Review:

1. Angwin, Julia; Larson, Jeff; Mattu, Surya; Kirchner, Lauren (2016). "Machine Bias: There’s software used across the country to predict future criminals. And it’s biased against blacks." ProPublica, May 23, 2016. https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing


2. Hao, Karen; Stray, Jonathan (2019). "Can you make AI fairer than a judge? Play our courtroom algorithm game: The US criminal legal system uses predictive algorithms to try to make the judicial process less biased. But there’s a deeper problem." MIT Technology Review, October 17, 2019. https://www.technologyreview.com/2019/10/17/75285/ai-fairer-than-judge-criminal-risk-assessment-algorithm/

You will need to read these articles for the sufficient background needed to successfully complete this homework assignment. It may also be useful to refer back to content covered during the mid-semester lecture on ethics: [Lecture 11: EthiCS](https://edstem.org/us/courses/26506/lessons/46044/slides/263725).

### Data description

In this assignment, we will be analyzing a portion of the data set used by ProPublica in their report. The data are found in the `compas.csv` file and the "data dictionary", which very briefly explains each variable in the data set, can be found in the `compas_datadict.csv` file.

<a id="part1"></a>

## <div class='exercise'>Question 1: Data exploration [26 pts]</div> 
    
[Return to contents](#contents)

The code provided below will load, preprocess, and split the COMPAS data into train and test sets for you.

In [ ]:
# Read in the data
compas_df = pd.read_csv("data/compas.csv")

# Inspect the original dataframe shape
print(
    "Prior to processing our data, the original COMPAS dataframe was "
    "shape:\n\n\t{}\n\nAnd, these were the first several observations:"
    .format(compas_df.shape)
)

# Inspect original a few observations prior to processing the attributes
display(compas_df.head())

# Process binary categorical variables
compas_df["sex"] = (compas_df["sex"] == "Male").astype(int)

# print(compas_df["c_charge_degree"].value_counts())
compas_df["felony"] = (compas_df["c_charge_degree"] == "F").astype(int)

# One-hot-encode the race variable
one_hot_df = pd.get_dummies(
    compas_df["race"], prefix="race", drop_first=False
)
compas_race_df = pd.concat(
    [compas_df.drop("race", axis=1), one_hot_df], axis=1
)

# Drop the categorical variables and leave n-1 race columns
compas_race_df = compas_race_df.drop(
    [
        "c_charge_degree",
        "c_charge_desc",
        "score_text",
        "race_Other",
    ],
    axis=1,
)

# Make 80-20 train-test split, stratifying on race
train_df, test_df = train_test_split(
    compas_race_df,
    train_size=0.8,
    stratify=compas_df["race"],
    random_state=50,
)

# Inspect training dataframe after processing and splitting the data
print(
    "\n\nAfter processing and splitting our data, our training and test"
    " data were of the shapes:\n\n\t{}\t{}\n\nAnd, these were the first"
    " several observations of the training set:"
    .format(train_df.shape, test_df.shape)
)
display(train_df.head())

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**1.1** What is the statistical objective of the COMPAS algorithm?  How is recidivism prediction used in courtrooms?

</div>

_Points:_ 7

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**1.2** Use the information in the provided `compas_datadict.csv` to answer the following question about the variables in the dataset:
    
* Which variables are raw data?  Which are pre-processed data? Justify your answer.
* Which is the target variable of the COMPAS algorithm?
* Which are output(s) of the COMPAS algorithm?
* Which should not be used in building our own model to predict recidivism? Why?

</div>

_Points:_ 6

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**1.3** Now we will do a little EDA. Code has been provided to generate a visualization that explores which **unprocessed** quantitative variables, including `decile_score` (**not** the derived binary variables or pre-processed versions of the available quantitative variables, for example), are related to `race`. Feel free to create additional plots of your own if you like.

Based on the visualization(s), report which 3 variables appear to have the most significant differences between Caucasians and African Americans. Briefly interpret your findings for each of those 3 variables, being certain to explain your justification for choosing each.

**NOTE:** As illustrated by [the required readings](#intro), reporting on the U.S. Criminal Justice system often concentrates on the disparate experiences between Caucasians and African Americans, thus we have focused our approaches similarly. However, to ensure the efficacy of our models in Question 2 and Question 3, we must be certain to still keep observations for all racial groups included in our data. 
</div>

_Points:_ 7

In [ ]:
# List of unprocessed variables and decile score (e.g.,
# should not include any that are pre-processed in
# compas.csv)
chosen_cols = [
    "age",
    "priors_count",
    "juv_fel_count",
    "juv_misd_count",
    "juv_other_count",
    "length_of_stay",
    "decile_score"
]

# Generate indices for subsetting data based on racial group
aa_idx_train = np.where(train_df["race_African-American"]==1)[0]
cc_idx_train = np.where(train_df["race_Caucasian"]==1)[0]
non_aa_cc_idx_train = np.where(
    np.all(
        [
            train_df["race_Caucasian"]==0,
            train_df["race_African-American"]==0
        ],
        axis=0
    )
)[0]

# Generate plots as required
fig, axes = plt.subplots(4, 2, figsize=(11, 9.5))

labels = ["African-American", "Caucasian", "All Other Groups"]
indices = [aa_idx_train, cc_idx_train, non_aa_cc_idx_train]
colors = ["tab:blue", "tab:orange", "none"]
edgecolors = ["none", "none", "k"]
alphas = [0.4, 0.4, 0.2]

plt.suptitle(
    "Marginal distributions of unprocessed quantitative attributes\n"
    "and decile score by racial group in the training data",
    fontsize=15,
    y=1,
)

for i, (col, ax) in enumerate(zip(chosen_cols, axes.flatten())):
    for j, (idx, label, color, alpha, edgecolor) in enumerate(
        zip(indices, labels, colors, alphas, edgecolors)
    ):

        min_val = train_df[col].min()
        max_val = train_df[col].max()
        
        # Plot as barplot if variable has small range of discrete values
        if max_val < 40:
            n_obs = len(idx) 
            val_counts = train_df[col].iloc[idx].value_counts()
            vals = val_counts.index
            counts = val_counts.values
            ax.bar(
                vals,
                counts / n_obs,
                alpha=alpha,
                label=label,
                color=color,
                edgecolor=edgecolor,
            )
            ax.set_xticks(
                range(min_val, max_val+1) if max_val<=20
                else range(min_val, max_val+1, 2)
            )
        # Plot as histogram if sufficiently large range of values
        else:
            ax.hist(
                train_df[col].iloc[idx],
                alpha=alpha,
                label=label,
                color=color if not "none" else None,
                histtype="step" if color=="none" else "bar",
                edgecolor=edgecolor,
                density=True,
                bins=15,
            )
        if i%2==0:
            ax.set_ylabel("Density", fontsize=12)
        if i==0:
            ax.legend(fontsize=12)
        ax.set_xlabel(col, fontsize=12)
        ax.grid(":", alpha=0.2)
axes[3,1].axis("off")

plt.tight_layout()

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**1.4** With respect to these 3 chosen variables, how could bias in the data or data collection be impacting or causing these differences?

</div>

_Points:_ 6

_Type your answer here, replacing this text._

<!-- END QUESTION -->

---

<a id="part2"></a>

## <div class="exercise">Question 2: Baseline modeling [21 pts]</div> 
    
[Return to contents](#contents)

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**2.1**  In this section we train a well-tuned, $LASSO$-like regularized logistic regression model to predict recidivism (i.e. `two_year_recid` is your response variable). We'll use the following predictors: `age`, `priors_1`, `priors_234`, `priors_5plus`, `juv_fel_1plus`, `juv_misd_1plus`, `juv_other_1plus`, `charge_any_drug`, `charge_any_violence_aggression`, `charge_any_theft`, as well as the one-hot-encoded predictors for `c_charge_degree`, `sex`, and `race`. 
    
Inspect the resulting coefficients and interpret what this model is saying about the relationship of `two_year_recid` to `race`.

</div>

_Points:_ 10

In [ ]:
# Separate predictors and response
predictor_list = [
    "age",
    "sex",
    "priors_1",
    "priors_234",
    "priors_5plus",
    "juv_fel_1plus",
    "juv_misd_1plus",
    "juv_other_1plus",
    "charge_any_drug",
    "charge_any_violence_aggression",
    "charge_any_theft",
    "felony",
    "race_African-American",
    "race_Asian",
    "race_Caucasian",
    "race_Hispanic",
    "race_Native American",
]

response_var = "two_year_recid"

X_train = train_df[predictor_list]
X_test = test_df[predictor_list]

y_train = train_df[response_var].values
y_test = test_df[response_var].values

# Inspect shape of resulting design matrix
print(
    "The shapes of our resulting X and y train and test sets "
    "are:\n\n\tX_train\t{}\n\ty_train\t{}\n\n\tX_test\t{}\n\t"
    "y_test\t{}\n".format(
        X_train.shape, y_train.shape, X_test.shape, y_test.shape,
    )
)

# Scale data to X_train, this is required for regularization
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the model
logit_model = LogisticRegressionCV(
    penalty="l1",
    solver="liblinear",
    max_iter=1000,
    random_state=109,
)
logit_model.fit(X_train_scaled, y_train)

# Print Coefficients
print(
    "The coefficients for the logistic regression model are:"
)
display(
    pd.DataFrame(
        index=["intercept"] + list(X_train.columns),
        data={
            "coefficients": np.hstack(
                [logit_model.intercept_, logit_model.coef_[0]]
            )
        },
    )
)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**2.2**  The code provided below reports the following metrics on the **test set**:

1. **Overall model accuracy score**


2. **False positive rates (FPR)** for each of the two groups (please note that a "positive" here is `two_year_recid==1`)
    - `African-American`
    - `Cacausian`
    
3. **False negative rates (FNR)** for each of the two groups:
    - `African-American`
    - `Caucasian`

4. **Ratios of error rates**:
    - Ratio of the FPR between `African-American` and `Caucasian` groups
    - Ratio of the FNR between `African-American` and `Caucasian` groups
    - These ratios can be expressed as:

$$\frac{FPR(AA)}{FPR(CC)} \; \text{and} \; \frac{FNR(AA)}{FNR(CC)}$$


What do you observe? Is there any disparity in the $FPR$ and $FNR$ for the two groups? What are the implications of your findings? Explain your findings in 3-6 sentences.

</div>

_Points:_ 11

In [ ]:
# Function to report required metric by model in Q2.2 and Q3.2
def evaluate_model(model, X_values, y_true, aa_idx, cc_idx):
    """Generate accuracy and error rate metrics for model
    
    Generates a set of test metrics for an input model, including
    model accuracy, FPR and FNR for both African-American and
    Caucasian racial groups, as well as FPR and FNR ratios
    defined as FPR(AfAm)/FPR(Caucasion) and 
    FNR(AfAm)/FNR(Caucasion)
    
    :param model: estimator, fitted scikit-learn classification
                  estimator such as LogisticRegression,
                  DecisionTreeClassifier, etc.
    :param X_values: np.ndarray, X matrix containing observations
                   for prediction and evaluation
    :param y_true: np.ndarray, true response classes
                   corresponding to each observation in the X
                   matrix
    :param aa_idx: np.ndarray, array of indices corresponding to
                   each African-American observation in the X
                   matrix
    :param cc_idx: np.ndarray, array of indices corresponding to
                   each Caucasion observation in the X matrix
    :returns: dict, resulting metrics including model accuracy,
              FPR and FNR for both African-American and Caucasian
              racial groups, as well as FPR and FNR ratios
    """
    y_pred = model.predict(X_values)
    model_accuracy = model.score(X_values, y_true)

    confusion_afam = dict(
        zip(
            ["tn","fp","fn","tp"],
            confusion_matrix(
                y_true[aa_idx], y_pred[aa_idx]
            ).ravel(),
        )
    )
    confusion_caucasian = dict(
        zip(
            ["tn","fp","fn","tp"],
            confusion_matrix(
                y_true[cc_idx], y_pred[cc_idx]
            ).ravel(),
        )
    )

    fpr_afam = (
        confusion_afam["fp"] / (
            confusion_afam["fp"] + confusion_afam["tn"]
        )
    )
    fnr_afam = (
        confusion_afam["fn"] / (
            confusion_afam["fn"] + confusion_afam["tp"]
        )
    )

    fpr_caucasian = (
        confusion_caucasian["fp"] / (
            confusion_caucasian["fp"] + confusion_caucasian["tn"]
        )
    )
    fnr_caucasian = (
        confusion_caucasian["fn"] / (
            confusion_caucasian["fn"] + confusion_caucasian["tp"]
        )
    )
    
    fpr_ratio = fpr_afam/fpr_caucasian
    fnr_ratio = fnr_afam/fnr_caucasian

    return dict(
        zip(
            [
                "Model Accuracy",
                "FPR African-American",
                "FNR African-American",
                "FPR Caucasian",
                "FNR Caucasian",
                "FPR Ratio",
                "FNR Ratio"
            ],
            [
                model_accuracy,
                fpr_afam,
                fnr_afam,
                fpr_caucasian,
                fnr_caucasian,
                fpr_ratio,
                fnr_ratio
            ],
        )
    )

In [ ]:
# Save the indices for each of the two races on the test set
aa_idx_test = np.where(test_df["race_African-American"]==1)[0]
cc_idx_test = np.where(test_df["race_Caucasian"]==1)[0]

results_logit_model = evaluate_model(
    logit_model,
    X_test_scaled,
    y_test,
    aa_idx_test,
    cc_idx_test,
)
print(
    "The accuracy and error rates for the model on the "
    "test set are:\n\n\t\t\tmodel with race\n"
)
    
for key, value in results_logit_model.items():
    print(f"{key:20}\t{value:.4f}")

_Type your answer here, replacing this text._

<!-- END QUESTION -->

---

<a id="part3"></a>

## <div class='exercise'>Question 3: Predicting without using `race` [19 pts]</div> 
    
[Return to contents](#contents)

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**3.1** Now we'll fit another well-tuned, $LASSO$-like regularized logistic regression model and report the sames metrics we did in Question 2, but this time we will fit the model **without** using the `race` predictors.

How do these metrics compare to those from the model that did include the `race` predictors?

</div>

_Points:_ 9

In [ ]:
# Drop the race columns, which are the last 5 columns in the df
X_train_scaled_no_race = X_train_scaled[:, :-5]
X_test_scaled_no_race = X_test_scaled[:, :-5]

# Fit the model
logit_model_no_race = LogisticRegressionCV(
    penalty="l1",
    solver = "liblinear",
    max_iter=1000,
    random_state=109
)
logit_model_no_race.fit(X_train_scaled_no_race, y_train)

# Print Coefficients
print(
    "The coefficients for the logistic regression model "
    "excluding race are:"
)

display(
    pd.DataFrame(
        index=["intercept"] + list(X_train.columns[:-5]),
        data={
            "coefficients": np.hstack(
                [
                    logit_model_no_race.intercept_,
                    logit_model_no_race.coef_[0],
                ]
            )
        },
    )
)

In [ ]:
# evaluate and report results
results_logit_no_race = evaluate_model(
    logit_model_no_race,
    X_test_scaled_no_race,
    y_test,
    aa_idx_test,
    cc_idx_test,
)

print(
    "The accuracy and error rates for the model on the "
    "test set are:\n\n\t\t\tmodel with race\t\twithout race\n"
)
    
for (key, value), value2 in zip(
    results_logit_model.items(), results_logit_no_race.values()
):
    print(f"{key:20}\t{value:.4f}\t\t\t{value2:.4f}")

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**3.2**  Given your exploration and modeling of the data, should a predictive tool be trusted to be unbiased even if it doesn’t explicitly use a variable such as race to predict future crime?  Why or why not? Give careful consideration to why the bias is still occurring or not still occurring in your results and what the real-life implications/effects might be for such a tool.

</div>

_Points:_ 10

_Type your answer here, replacing this text._

<!-- END QUESTION -->

---

<a id="part4"></a>

## <div class='exercise'>Question 4: Classification thresholds and fairness [34 pts]</div> 

Below is code that plots the Receiver Operating Characteristic (ROC) curves from the question 3 model for two racial groups: `African-American` and `Caucasian` on the **training set**. In question 4, we will explore the false positice and false negative rates of this model further.
[Return to contents](#contents)

In [ ]:
# generate predicted probabilities for training observations
preds_proba = logit_model_no_race.predict_proba(
    X_train_scaled_no_race
)[:,1]


# Generate AUC score separately for African-American and 
# Caucasian observations 
auc_score_aa = roc_auc_score(
    y_train[aa_idx_train], preds_proba[aa_idx_train]
)
auc_score_cc = roc_auc_score(
    y_train[cc_idx_train], preds_proba[cc_idx_train]
)

# Generate ROC curve values separately for African-American 
# and Caucasian observations
fpr_aa, tpr_aa, thresholds_aa = roc_curve(
    y_train[aa_idx_train], preds_proba[aa_idx_train]
)
fpr_cc, tpr_cc, thresholds_cc = roc_curve(
    y_train[cc_idx_train], preds_proba[cc_idx_train]
)


# Plot ROC curves
plt.figure(figsize=(9, 5.25))
plt.plot(
    fpr_aa,
    tpr_aa,
    label=f"African-American\n$AUC={auc_score_aa:.3f}$",
)
plt.plot(
    fpr_cc,
    tpr_cc,
    label=f"Caucasian\n$AUC={auc_score_cc:.3f}$",
)
plt.plot([0, 1], [0, 1], color="k", linestyle=":", alpha=0.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate ($FPR$)", fontsize=12)
plt.ylabel("True Positive Rate ($TPR$)", fontsize=12)
plt.title(
    "Receiver operating characteristic curves by racial group\n"
    "based on training observations",
    fontsize=16
)
plt.legend(fontsize=12, edgecolor="k")
plt.grid(":", alpha=0.4)
plt.tight_layout()

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**4.1** The default threshold used for determining the classification metrics analyzed in Question 2 and Question 3 was $\hat{p}=0.5$.  Choose a new single threshold for the model from question 3 that will reduce the bias between these two racial groups as measured by the "Ratios of Error Rates" (as defined by the formulas shown in Question 2). Be certain to justify your choice and interpet how this threshold choice affects your model results.

**HINT:** There is no one correct answer here. However, to arrive at an answer you can support with an evidence-based justification, you will likely need to inspect the "Ratio of $FPR$", "Ratio of $FNR$", and accuracy trends across varying classification thresholds. There should be no need to refit your model from question 3 to accomplish this task. You can use the code provide to help make your decision, but again, feel free to create your own additional plots if you like.

</div>

_Points:_ 7

In [ ]:
# Function to report required metrics by model and threshold
def evaluate_model_threshold(
    model, X_values, y_true, aa_idx, cc_idx, threshold
):
    """Generate metrics for model at a given threshold
    
    Generates a set of test metrics for an input model, including
    model accuracy and FPR and FNR for all observations and both
    African-American and Caucasian racial groups individually, as
    well as FPR and FNR ratios defined as FPR(AfAm)/FPR(Caucasion)
    and FNR(AfAm)/FNR(Caucasion)
    
    :param model: estimator, fitted scikit-learn classification
                  estimator such as LogisticRegression,
                  DecisionTreeClassifier, etc.
    :param X_values: np.ndarray, X matrix containing observations
                   for prediction and evaluation
    :param y_true: np.ndarray, true response classes
                   corresponding to each observation in the X
                   matrix
    :param aa_idx: np.ndarray, array of indices corresponding to
                   each African-American observation in the X
                   matrix
    :param cc_idx: np.ndarray, array of indices corresponding to
                   each Caucasion observation in the X matrix
    :param threshold: float, classification threshold, 0 to 1,
                      at which to evaluate model
    :returns: dict, resulting metrics including model accuracy,
              FPR, and FNR for all observations and both
              African-American and Caucasian racial groups, as
              well as FPR and FNR ratios
    """
    y_pred_proba = model.predict_proba(X_values)[:,1]
    y_preds = np.where(y_pred_proba >= threshold, 1, 0)
    model_accuracy = accuracy_score(y_preds, y_true)
    model_accuracy_afam = accuracy_score(
        y_preds[aa_idx], y_true[aa_idx]
    )
    model_accuracy_caucasian = accuracy_score(
        y_preds[cc_idx], y_true[cc_idx]
    )
    
    confusion_overall = dict(
        zip(
            ["tn","fp","fn","tp"],
            confusion_matrix(
                y_true, y_preds
            ).ravel(),
        )
    )

    confusion_afam = dict(
        zip(
            ["tn","fp","fn","tp"],
            confusion_matrix(
                y_true[aa_idx], y_preds[aa_idx]
            ).ravel(),
        )
    )
    confusion_caucasian = dict(
        zip(
            ["tn","fp","fn","tp"],
            confusion_matrix(
                y_true[cc_idx], y_preds[cc_idx]
            ).ravel(),
        )
    )

    fpr_all = (
        confusion_overall["fp"] / (
            confusion_overall["fp"] + confusion_overall["tn"]
        )
    )
    fnr_all = (
        confusion_overall["fn"] / (
            confusion_overall["fn"] + confusion_overall["tp"]
        )
    )

    
    fpr_afam = (
        confusion_afam["fp"] / (
            confusion_afam["fp"] + confusion_afam["tn"]
        )
    )
    fnr_afam = (
        confusion_afam["fn"] / (
            confusion_afam["fn"] + confusion_afam["tp"]
        )
    )

    fpr_caucasian = (
        confusion_caucasian["fp"] / (
            confusion_caucasian["fp"] + confusion_caucasian["tn"]
        )
    )
    fnr_caucasian = (
        confusion_caucasian["fn"] / (
            confusion_caucasian["fn"] + confusion_caucasian["tp"]
        )
    )
    
    with np.errstate(all="raise"):
        try:
            fpr_ratio = fpr_afam/fpr_caucasian
        except:
            fpr_ratio = 0

        try:
            fnr_ratio = fnr_afam/fnr_caucasian
        except:
            fnr_ratio = 0

    return dict(
        zip(
            [
                "threshold",
                "model_accuracy",
                "model_accuracy_afam",
                "model_accuracy_caucasian",
                "fpr_all",
                "fnr_all",
                "fpr_afam",
                "fnr_afam",
                "fpr_caucasian",
                "fnr_caucasian",
                "fpr_ratio",
                "fnr_ratio"
            ],
            [
                threshold,
                model_accuracy,
                model_accuracy_afam,
                model_accuracy_caucasian,
                fpr_all,
                fnr_all,
                fpr_afam,
                fnr_afam,
                fpr_caucasian,
                fnr_caucasian,
                fpr_ratio,
                fnr_ratio
            ],
        )
    )

In [ ]:
# create list of 100 thresholds, p=[0, 1]
t_vals = np.arange(0, 1.01, 0.01)

# instantiate dict for storing results at each threshold
results_dict_train, results_dict_test = {}, {}

# generate train and test results at each threshold and store to dicts
for i, t in enumerate(t_vals):
    result_train = evaluate_model_threshold(
        logit_model_no_race,
        X_train_scaled_no_race,
        y_train, 
        aa_idx_train,
        cc_idx_train,
        threshold=t,
    )    
    result_test = evaluate_model_threshold(
        logit_model_no_race,
        X_test_scaled_no_race,
        y_test, 
        aa_idx_test,
        cc_idx_test,
        threshold=t,
    )    

    for key, val in result_train.items():
        # use dict.setdefault method to append val list for each
        # key. For more info on this use of the method...
        # see here: https://docs.python.org/3/library/
        #           stdtypes.html#dict.setdefault
        # and here: https://stackoverflow.com/a/26367880
        results_dict_train.setdefault(key, []).append(val)

    for key, val in result_test.items():
        results_dict_test.setdefault(key, []).append(val)


In [ ]:
def plot_threshold(threshold):
    # instantiate lists for plotting axes
    key_list = ["fpr_ratio", "fnr_ratio", "model_accuracy"]
    title_list = ["$FPR$ ratio", "$FNR$ ratio", "Model accuracy"]

    # generate plot
    fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True)

    for i, (ax, key, title) in enumerate(
        zip(axes, key_list, title_list)
    ):
        ax.plot(results_dict_train["threshold"], results_dict_train[key])
        ax.set_ylabel(
            "Ratio" if "ratio" in title else "Accuracy",
            fontsize=12
        )
        ax.axvline(
            threshold,
            color="k",
            linestyle="--",
            label="Proposed threshold: $\hat{{p}}={:.2f}$".format(
                threshold
            ),
        )
        ax.axhline(
            results_dict_train[key][50],
            color="tab:red",
            linestyle=":",
            label="Metrics values at default threshold $\hat{{p}}=0.5$"
        )
        ax.set_title(title)
        ax.grid(":", alpha=0.4)
        if i==0:
            ax.legend(edgecolor="k", fontsize=11)
        if i==2:
            ax.set_xlabel("Classification threshold", fontsize=12)

    plt.suptitle(
        "Ratios of error rates "
        "$\left( \; \\frac{FPR(AA)}{FPR(CC)} \; and \; "
        "\\frac{FNR(AA)}{FNR(CC)} \; \\right)$ "
        "and model accuracy\nby classification threshold "
        "on the TRAINING data",
        fontsize=15,
    )
    plt.tight_layout()

    # print summary results
    group_list = ["Caucasian", "African-American"]
    var_list = ["caucasian", "afam"]
    threshold_indices = [int(threshold*1E+2)]*2

    print(
        "Naive model accuracy results, predicting all observations "
        "as Class 0\nbased on {:.4f} overall proportion Class 1 in "
        "training observations:\n\n\t\t\t\tTrain\t\tTEST\n"
        "\tCaucasian\t\t{:.4f}\t\t{:.4f}\n"
        "\tAfrican-American\t{:.4f}\t\t{:.4f}\n"
        "\tAll Observations\t{:.4f}\t\t{:.4f}\n".format(
            sum(y_train)/len(y_train),
            1-sum(y_train[cc_idx_train])/len(y_train[cc_idx_train]),
            1-sum(y_test[cc_idx_test])/len(y_test[cc_idx_test]),
            1-sum(y_train[aa_idx_train])/len(y_train[aa_idx_train]),
            1-sum(y_test[aa_idx_test])/len(y_test[aa_idx_test]),
            1-sum(y_train)/len(y_train),
            1-sum(y_test)/len(y_test),
        )
    )

    for group, var, t_idx in zip(
        group_list, var_list, threshold_indices
    ):
        print(
            "{} results with proposed threshold p={:.2f}:\n\n"
            "\t\t\tTrain\t\tTEST\n"
            "\tFPR\t\t{:.4f}\t\t{:.4f}\n"
            "\tFNR\t\t{:.4f}\t\t{:.4f}"
            "\n\tAccuracy\t{:.4f}\t\t{:.4f}\n"
            .format(
                group,
                results_dict_train['threshold'][t_idx],
                results_dict_train[f"fpr_{var}"][t_idx],
                results_dict_test[f"fpr_{var}"][t_idx],
                results_dict_train[f"fnr_{var}"][t_idx],
                results_dict_test[f"fnr_{var}"][t_idx],
                results_dict_train[f"model_accuracy_{var}"][t_idx],
                results_dict_test[f"model_accuracy_{var}"][t_idx],
            )
        )

    print(
        "Overall results (all racial groups) with proposed "
        "threshold p={:.2f}:\n\n\t\t\tTrain\t\tTEST\n"
        "\tFPR\t\t{:.4f}\t\t{:.4f}\n\tFNR\t\t{:.4f}\t\t{:.4f}"
        "\n\tAccuracy\t{:.4f}\t\t{:.4f}\n".format(
            results_dict_train['threshold'][t_idx],
            results_dict_train[f"fpr_all"][t_idx],
            results_dict_test[f"fpr_all"][t_idx],
            results_dict_train[f"fnr_all"][t_idx],
            results_dict_test[f"fnr_all"][t_idx],
            results_dict_train[f"model_accuracy"][t_idx],
            results_dict_test[f"model_accuracy"][t_idx],
        )
    )

_Type your answer here, replacing this text._

In [ ]:
# set threshold for plotting and evaluating
threshold = ...
plot_threshold(threshold)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**4.2** A second approach to reducing bias is to use different thresholds for the different racial groups to better ensure that the individual groups have similar false positive and false negative rates.  Choose a pair of thresholds (one for the `African-American` group and one for the `Caucasian` group) that improves the group bias (while still taking accuracy into consideration). Be certain to justify your choices and interpet how these threshold choices affect your model results.

**HINT:** To arrive at an answer you can support with an evidence-based justification, you will likely need to inspect $FPR$, $FNR$, and accuracy trends across varying classification thresholds for each group (`African-American` and `Caucasian`) individually. There should be no need to refit your model from Question 3 to accomplish this task. Again, the plotting code provided can be used to help decide on a pair of thresholds.

</div>

_Points:_ 7

In [ ]:
def plot_threshold_pair(t_cc, t_afam):
    # instantiate lists for plotting axes
    group_list = ["Caucasian", "African-American"]
    var_list = ["caucasian", "afam"]
    threshold_list = [t_cc, t_afam]


    # Generate plots
    fig, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 4.5))
    plt.suptitle(
        '$FNR$, $FPR$, and accuracy trend and proposed "balanced"\n'
        "classification thresholds by racial group using "
        "the TRAINING data",
        fontsize=16,
        y=1,
    )

    for i, (group, var, threshold, ax) in enumerate(
        zip(group_list, var_list, threshold_list, axes)
    ):

        ax.plot(
            results_dict_train["threshold"],
            results_dict_train[f"fnr_{var}"],
            label="FNR",
        )
        ax.plot(
            results_dict_train["threshold"],
            results_dict_train[f"fpr_{var}"],
            label="FPR",
        )
        ax.plot(
            results_dict_train["threshold"],
            results_dict_train[f"model_accuracy_{var}"],
            "k-",
            label="Accuracy",
        )
        ax.axvline(
            threshold,
            color="k",
            linestyle="--",
            label="Proposed\nthreshold",
        )
        ax.set_title(f"{group} training observations", fontsize=14)
        ax.set_xlabel("Classification threshold", fontsize=12)
        ax.grid(":", alpha=0.4)

        if i==0:
            ax.legend(edgecolor="k", fontsize=12, loc=(.66, .11))
            ax.set_ylabel("Resulting metric value", fontsize=12)

    plt.tight_layout()

    # print summary results
    threshold_indices = [int(t_cc*1E+2), int(t_afam*1E+2)]

    for group, var, t_idx in zip(
        group_list, var_list, threshold_indices
    ):
        print(
            "{} results with proposed threshold p={:.2f}:\n\n"
            "\t\t\tTrain\t\tTEST\n"
            "\tFPR\t\t{:.4f}\t\t{:.4f}\n"
            "\tFNR\t\t{:.4f}\t\t{:.4f}"
            "\n\tAccuracy\t{:.4f}\t\t{:.4f}\n"
            .format(
                group,
                results_dict_train['threshold'][t_idx],
                results_dict_train[f"fpr_{var}"][t_idx],
                results_dict_test[f"fpr_{var}"][t_idx],
                results_dict_train[f"fnr_{var}"][t_idx],
                results_dict_test[f"fnr_{var}"][t_idx],
                results_dict_train[f"model_accuracy_{var}"][t_idx],
                results_dict_test[f"model_accuracy_{var}"][t_idx],
            )
        )

_Type your answer here, replacing this text._

In [ ]:
# identify balanced thresholds
# thresholds for Caucasians 
t_cc = ...

# threshold for African Americans
t_afam = ...

plot_threshold_pair(t_cc, t_afam)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**4.3** Comment on [the fairness](<https://en.wikipedia.org/wiki/Fairness_(machine_learning)>) of the two methods in Question 4.1 and Question 4.2 from two different perspectives: the fairness of each group (called "group fairness") and the fairness for an individual defendant (called "individual fairness"). 

</div>

_Points:_ 10

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class='exercise-r'>

**4.4** Changing the thresholds can *reduce* bias between the two classes, but it can also affect model accuracy. We want our model to be **accurate** but also **fair**.  What can be done to balance these two criteria?  Write down your strategy to overcome this difficulty. Please limit your response to at most 150 words.  

</div>

_Points:_ 10

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**This concludes HW7. Thank you!**

[Return to contents](#contents)